In [ ]:
!nvidia-smi

Tue Aug 17 14:27:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#**TIẾN HÀNH DOWNLOAD DATA VỀ**

In [ ]:
# Step 1. Mount drive CỦA CÁ NHÂN
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import gdown
%cd /content/gdrive/My\ Drive
!mkdir Data_CS114
%cd /content/gdrive/My\ Drive/Data_CS114

/content/gdrive/My Drive
/content/gdrive/My Drive/Data_CS114


In [ ]:
!gdown --id 1k2YUBJISA_0hueQTYnyA_MRONSjR1Mnw #Trainimg.zip
!gdown --id 1m77ZWzYtKyed56-mVSDHSSiPFV7QDMJh #Trainlabel.zip
# !gdown --id 1FRg4q4qTi2ImfpMxE5EnLeDRCHNxo13C #Testimg.zip
# !gdown --id 1gDtyugP41svdzTfWaWyysWC4Co3X0rIs #Testlabel.zip


Downloading...
From: https://drive.google.com/uc?id=1k2YUBJISA_0hueQTYnyA_MRONSjR1Mnw
To: /content/gdrive/My Drive/Data_CS114/trainimg.zip
3.63GB [01:33, 39.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1m77ZWzYtKyed56-mVSDHSSiPFV7QDMJh
To: /content/gdrive/My Drive/Data_CS114/trainlabel.zip
100% 1.10M/1.10M [00:00<00:00, 5.16MB/s]
Downloading...
From: https://drive.google.com/uc?id=1FRg4q4qTi2ImfpMxE5EnLeDRCHNxo13C
To: /content/gdrive/My Drive/Data_CS114/testimg.zip
1.49GB [00:52, 28.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1gDtyugP41svdzTfWaWyysWC4Co3X0rIs
To: /content/gdrive/My Drive/Data_CS114/testlabel.zip
100% 332k/332k [00:00<00:00, 2.38MB/s]


In [ ]:
!mkdir Train
# !mkdir Test

In [ ]:

!unzip trainimg.zip -d "/content/gdrive/MyDrive/Data_CS114/Train"
# !unzip testimg.zip -d "/content/gdrive/MyDrive/Data_CS114/Test"
!unzip trainlabel.zip -d "/content/gdrive/MyDrive/Data_CS114/Train"
# !unzip testlabel.zip -d "/content/gdrive/MyDrive/Data_CS114/Test"

Streaming output truncated to the last 5000 lines.
  inflating: /content/gdrive/MyDrive/Data_CS114/Train/trainimg/Yoloimage2111.jpg  
  inflating: /content/gdrive/MyDrive/Data_CS114/Train/trainimg/Yoloimage2112.jpg  
  inflating: /content/gdrive/MyDrive/Data_CS114/Train/trainimg/Yoloimage2113.jpg  
  inflating: /content/gdrive/MyDrive/Data_CS114/Train/trainimg/Yoloimage2114.jpg  
  inflating: /content/gdrive/MyDrive/Data_CS114/Train/trainimg/Yoloimage2115.jpg  
  inflating: /content/gdrive/MyDrive/Data_CS114/Train/trainimg/Yoloimage2116.jpg  
  inflating: /content/gdrive/MyDrive/Data_CS114/Train/trainimg/Yoloimage2117.jpg  
  inflating: /content/gdrive/MyDrive/Data_CS114/Train/trainimg/Yoloimage2118.jpg  
  inflating: /content/gdrive/MyDrive/Data_CS114/Train/trainimg/Yoloimage2119.jpg  
  inflating: /content/gdrive/MyDrive/Data_CS114/Train/trainimg/Yoloimage212.jpg  
  inflating: /content/gdrive/MyDrive/Data_CS114/Train/trainimg/Yoloimage2120.jpg  
  inflating: /content/gdrive/MyDrive/

In [ ]:
# import all the tools we need
import urllib
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import DataLoader, Dataset
import os 
from PIL import Image
import random
import xml.etree.ElementTree as ET
import time
import requests
from google.colab.patches import cv2_imshow
import matplotlib.patches as patches

In [ ]:
# path of images directory
img_train_path = '/content/gdrive/MyDrive/Data_CS114/Train/trainimg'
# img_test_path = '/content/gdrive/MyDrive/Data_CS114/Test/testimg'

# path of xml files directory
label_train_path = '/content/gdrive/MyDrive/Data_CS114/Train/train_label'
# label_test_path = '/content/gdrive/MyDrive/Data_CS114/Test/test_label'

# List of Image file name 
img_train_list =list(sorted(os.listdir(img_train_path)))
# img_test_list = list(sorted(os.listdir(img_test_path)))
label_train_list= list(sorted(os.listdir(label_train_path)))
# label_test_list= list(sorted(os.listdir(label_test_path)))

# How many image files?
# print('There are total {} images train and {} images test.'.format(len(img_train_list),len(img_test_list)))

#**HÀM ĐỌC LABEL VÀ VẼ BOUNDING BOX**

In [ ]:
# Helper function for read the data (label and bounding boxes) from xml file 
def read_label(file_name, label_dir):
    """
    Function used to get the bounding boxes and labels from the xml file
    Input:
        file_name: image file name
        xml_dir: directory of xml file
    Return:
        bbox : list of bounding boxes
        labels: list of labels
    """
    bbox = []
    labels = []
    
    label_path = os.path.join(label_dir, file_name[:-3]+'xml')
    tree = ET.parse(label_path)
    root = tree.getroot()
    for boxes in root.iter('object'):
        ymin = int(boxes.find("bndbox/ymin").text)
        xmin = int(boxes.find("bndbox/xmin").text)
        ymax = int(boxes.find("bndbox/ymax").text)
        xmax = int(boxes.find("bndbox/xmax").text)
        label = boxes.find('name').text
        bbox.append([xmin,ymin,xmax,ymax])
        if label == 'Correct':
            label_idx = 1
        elif label == 'No-mask':
          label_idx = 2
        elif label == 'Incorrect':
            label_idx = 3
        labels.append(label_idx)
        
    return bbox, labels

# help function for drawing bounding boxes on image
def draw_boxes(img, boxes,labels, thickness=3):
    """
    Function to draw bounding boxes
    Input:
        img: array of img (h, w ,c)
        boxes: list of boxes (int)
        labels: list of labels (int)
    
    """
    font =  cv2.FONT_HERSHEY_SIMPLEX
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    for box,label in zip(boxes,labels):
        box = [int(x) for x in box]
        if label == 3:
          color = (255,0,255) #pink
          text = 'Incorrect'
        elif label == 2:
          color = (0,0,225) # red
          text = 'Nomask'
        elif label == 1:
          color = (0,255,0) # green
          text = 'Correct'
        cv2.putText(img,text,(box[0],box[1]), font, 1 ,color, thickness)
        cv2.rectangle(img, (box[0],box[1]),(box[2],box[3]),color,thickness)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#**THIẾT LẬP DATASET**

In [ ]:
class image_dataset(Dataset):
    def __init__(self, image_list, image_dir, label_train_dir):
        self.image_list = image_list
        self.image_dir = image_dir
        self.xml_dir = label_train_dir
        self.classes = [_, 'Correct','No-mask','Incorrect']
    def __getitem__(self, idx):
        """
        Load the image
        """
        img_name = self.image_list[idx]
        img_path = os.path.join(self.image_dir, img_name)
        img = Image.open(img_path).convert('RGB')
        img = transforms.ToTensor()(img)

        """
        build the target dict
        """
        bbox, labels = read_label(img_name, self.xml_dir)
        boxes = torch.as_tensor(bbox, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        
        area = (boxes[:,2] - boxes[:,0]) * (boxes[:,3] - boxes[:,1])
        area = torch.as_tensor(area, dtype=torch.float32)
        iscrowd = torch.zeros((len(bbox),), dtype=torch.int64)
        
        target = {}
        
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([idx])
        target['area'] = area
        target['iscrowed'] = iscrowd
        return img , target
                    
    def __len__(self):
        return len(self.image_list)

In [ ]:
mask_dataset_train = image_dataset(img_train_list, img_train_path, label_train_path)

def collate_fn(batch):
    return tuple(zip(*batch))

mask_loader_train = DataLoader(mask_dataset_train,batch_size=10,shuffle=True,num_workers=4,collate_fn=collate_fn)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


#**THIẾT LẬP CÁC THAM SỐ**

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

In [ ]:
model = get_model_instance_segmentation(4).to(device)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

In [ ]:
params = [p for p in model.parameters() if p.requires_grad]
#optimizer = torch.optim.Adam(params, lr=0.01)
optimizer = torch.optim.SGD(params, lr=0.005,momentum=0.9, weight_decay=0.0005)
#lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=3,gamma=0.1)
# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=3,gamma=0.1)
num_epochs=20

In [ ]:
# import torch
# torch.cuda.empty_cache()

#**TRAIN MODEL**

In [ ]:
# Main training function
loss_list = []

for epoch in range(num_epochs):
    print('Starting training....{}/{}'.format(epoch+1, num_epochs))
    loss_sub_list = []
    start = time.time()
    for images, targets in mask_loader_train:
        images = list(image.to(device) for image in images)
        targets = [{k:v.to(device) for k,v in t.items()} for t in targets]
        
        model.train()
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()
        loss_sub_list.append(loss_value)
        
        # update optimizer and learning rate
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        #lr_scheduler.step()
    end = time.time()
        
    #print the loss of epoch
    epoch_loss = np.mean(loss_sub_list)
    loss_list.append(epoch_loss)
    print('Epoch loss: {:.3f} , time used: ({:.1f}s)'.format(epoch_loss, end-start))

#**SAVE MODEL**

In [ ]:
torch.save(model.state_dict(),'./model_final_cs114.pth')